# Fitting BKT to students

Criar a seguinte estrutura de dados:
1. ID do aluno
2. ID do problema
3. Lista com soluções 1 a N
6. Lista com conceito mais predominante nas soluções 1 a N
5. Lista indicando se resolveu em 1 a N soluções


### Import libraries

In [2]:
#DB
from questions.models import UserLog
import psycopg2

# Helpers
import numpy as np
import pickle
import base64
from tqdm import tqdm_notebook as tqdm
import pandas as pd

# Preprocessing
from tokenizer import create_bag_of_words
from sklearn.feature_extraction.text import CountVectorizer

### Connect to DB

In [3]:
connection = psycopg2.connect(user = "machineteaching",
                                  password = "",
                                  host = "localhost",
#                                   port = "5432",
                                  database = "machineteaching")
connection.autocommit=True
cursor = connection.cursor()

#### Load base solutions to re-create vectorizer

In [4]:
def get_original_solutions():
    ## Cleaning database
    last_id = 132
    problems = Problem.objects.filter(id__gt=last_id)
    solutions_obj = Solution.objects.filter(problem__in=problems).update(ignore=True)
    print("Problems to be ignored: %d" % problems.count())

    problems = Problem.objects.filter(id__lte=last_id)
    # problems = Problem.objects.all()
    print("Problems to be used: %d" % problems.count())

    solutions_obj = Solution.objects.filter(problem__in=problems, ignore=False).order_by('id')
    # solutions_obj = Solution.objects.all().order_by('id')
    print("Solutions to be used: %d" % solutions_obj.count())

    docs_id = []
    questions = []
    solutions = []

    # Fill separated structures
    for sol in solutions_obj:
        docs_id.append(sol.id)
        questions.append(sol.problem.content)
        solutions.append(sol.content)

    print("Got %d documents" %(solutions_obj.count()))
    return solutions

#### Load model from database e transform new observations

In [5]:
def get_where_items(exp_id):
    """ Get where clause for experiment """
    cols = ["vectorizer", "min_df", "is_binary", "distance", "method", "dataset", "k", "model", "X"]
    query = "SELECT %s from experiments_solution where experiment_id = %s" % (", ".join(cols), exp_id) 
    cursor.execute(query)
    where_items = cursor.fetchall()
    return where_items

def recreate_vectorizer(solutions, where_items, exp_id):
    """ Recreate vectorizer from where clause conditions """
    v = eval(where_items[0][0])
    m = where_items[0][1]
    b = where_items[0][2]
    dist = where_items[0][3]
    method = where_items[0][4]
    k = where_items[0][6]
    model_db = pickle.loads(base64.b64decode(where_items[0][7]))
    X = np.asarray(where_items[0][8])

    train_data_features, vectorizer, feature_names = create_bag_of_words(solutions, v, binary=b, min_df=m)
    
    return vectorizer

def get_model_and_vectorizer(exp_id=26):
    """ Recreate vectorizer and load saved model from database """
    # Recreate vectorizer
    where_items = get_where_items(exp_id)
    solutions = get_original_solutions()
    vectorizer = recreate_vectorizer(solutions, where_items, exp_id)
    
    # Load saved model
    model_db = pickle.loads(base64.b64decode(where_items[0][7]))
    model_db.n_jobs = 1
    
    return vectorizer, model_db

def transform(student, model_db, vectorize):
    """ Transform a new observation using the given vectorizer and model """
    train_data_features_student = vectorizer.transform(student).toarray()
    y_student = model_db.transform(train_data_features_student)
    return y_student

def get_skill(student_solution, vectorizer, model_db):
    """ Get argmax topic after transforming a new observation """
    y_student = transform(student_solution, model_db, vectorizer)
    top_skill = np.argmax(y_student, axis=1)
    return top_skill

In [6]:
vectorizer, model = get_model_and_vectorizer()

Problems to be ignored: 591
Problems to be used: 132
Solutions to be used: 54
Got 54 documents


/home/lmoraes/miniconda3/envs/machineteaching/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [124]:
user_blacklist = UserProfile.objects.filter(professor='')
logs_list = UserLog.objects.exclude(outcome='S').exclude(user__userprofile__in=user_blacklist).order_by("timestamp")
solution_list = logs_list.values_list('solution', flat=True)

In [125]:
skills = get_skill(solution_list, vectorizer, model)

In [126]:
%timeit
data = {}
for idx, log in enumerate(tqdm(logs_list)):
    # Data identification is a tuple containing student id and problem id
    student_id = log.user.pk
    problem_id = log.problem.pk
    data_id = (student_id, problem_id)
    
    # If there isn't anything concerning this id, add it to dict
    if not data_id in data.keys():
        data[data_id] = {
            "student_id": student_id,
            "problem_id": problem_id,
            "solutions": [log.solution],
            "outcomes": [log.outcome],
            "skills": [skills[idx]]
        }
    # If basic information is already there, just complete with solution info
    else:
        data[data_id]["solutions"].append(log.solution)
        data[data_id]["outcomes"].append(log.outcome)
        data[data_id]["skills"].append(skills[idx])

In [127]:
df = pd.DataFrame(data.values())

In [128]:
# Calculate if student learned in the N prediction
def success(row, N):
    if "P" in row["outcomes"][:N]:
        row["solved_in_%d" % N] = 1 # For correct
    else:
        row["solved_in_%d" % N] = 2 # For incorrect
    skill_str = [str(s) for s in set(row["skills"][:N])]
    row["skill_in_%d" % N] = "~".join(skill_str)
    return row

In [129]:
df = df.apply(success, args=(1,), axis=1).apply(success, args=(2,), axis=1).apply(success, args=(3,), axis=1)

In [130]:
with open("transaction_data.pkl", "wb") as pkl_file:
    pickle.dump(df, pkl_file)

In [16]:
df

,outcomes,problem_id,skills,solutions,student_id,solved_in_1,skill_in_1,solved_in_2,skill_in_2,solved_in_3,skill_in_3
0,"[F, F, P]",118,"[5, 5, 5]",[# Complete your function using this header\nd...,47,2,5,2,5,1,5
1,"[F, P]",118,"[5, 5]",[# Complete your function using this header\nd...,48,2,5,1,5,1,5
2,[P],118,[5],[# Complete your function using this header\nd...,51,1,5,1,5,1,5
3,[P],129,[5],[# Complete your function using this header\nd...,54,1,5,1,5,1,5
4,"[F, F, F, F, F, F, F, F]",118,"[5, 5, 5, 5, 5, 5, 5, 5]",[# Complete your function using this header\nd...,50,2,5,2,5,2,5
5,"[F, F, F, F, F, F, F, F, F, F, F, F, F, F, F, ...",120,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...",[# Complete your function using this header\nd...,51,2,11,2,11,2,11
6,[P],28,[5],[#Start your python function here\ndef max_of_...,56,1,5,1,5,1,5
7,[P],78,[5],[# Complete your function using this header\nd...,60,1,5,1,5,1,5
8,"[F, F, P]",120,"[9, 11, 11]",[# Complete your function using this header\nd...,47,2,9,2,9~11,1,9~11
9,"[F, F, F, F, F, F, F]",120,"[11, 11, 11, 11, 9, 9, 9]",[# Complete your function using this header\nd...,50,2,11,2,11,2,11


In [221]:
data_solved_in_1 = df[["solved_in_1", "student_id", "problem_id", "skill_in_1"]]
data_solved_in_1.to_csv("data_solved_in_1.csv", sep='\t', header=False, index=False)

In [216]:
data_solved_in_2 = df[["solved_in_2", "student_id", "problem_id", "skill_in_2"]]
data_solved_in_2.to_csv("data_solved_in_2.csv", sep='\t', header=False, index=False)

In [217]:
data_solved_in_3 = df[["solved_in_3", "student_id", "problem_id", "skill_in_3"]]
data_solved_in_3.to_csv("data_solved_in_3.csv", sep='\t', header=False, index=False)

In [224]:
data_solved_in_1_skill_5 = df[df["skill_in_1"] == '5'][["solved_in_1", "student_id", "problem_id", "skill_in_1"]]
data_solved_in_1_skill_5.to_csv("data_solved_in_1_skill_5.csv", sep='\t', header=False, index=False)

In [229]:
data_solved_in_2_skill_5 = df[df["skill_in_2"].str.contains('5')][["solved_in_2", "student_id", "problem_id", "skill_in_2"]]
data_solved_in_2_skill_5.to_csv("data_solved_in_2_skill_5.csv", sep='\t', header=False, index=False)

In [239]:
data_solved_in_3_skill_5 = df[df["skill_in_3"].str.contains('5')][["solved_in_3", "student_id", "problem_id", "skill_in_3"]]
data_solved_in_3_skill_5["skill_in_3"] = 5
data_solved_in_3_skill_5.to_csv("data_solved_in_3_skill_5.csv", sep='\t', header=False, index=False)

In [232]:
data_solved_in_1_skill_11 = df[df["skill_in_1"] == '11'][["solved_in_1", "student_id", "problem_id", "skill_in_1"]]
data_solved_in_1_skill_11.to_csv("data_solved_in_1_skill_11.csv", sep='\t', header=False, index=False)

In [235]:
data_solved_in_2_skill_11 = df[df["skill_in_2"].str.contains('11')][["solved_in_2", "student_id", "problem_id", "skill_in_2"]]
data_solved_in_2_skill_11["skill_in_2"] = 11
data_solved_in_2_skill_11.to_csv("data_solved_in_2_skill_11.csv", sep='\t', header=False, index=False)

In [237]:
data_solved_in_3_skill_11 = df[df["skill_in_3"].str.contains('11')][["solved_in_3", "student_id", "problem_id", "skill_in_3"]]
data_solved_in_3_skill_11["skill_in_3"] = 11
data_solved_in_3_skill_11.to_csv("data_solved_in_3_skill_11.csv", sep='\t', header=False, index=False)

In [242]:
data_solved_in_2

,solved_in_2,student_id,problem_id,skill_in_2
0,2,47,118,5
1,1,48,118,5
2,1,51,118,5
3,1,54,129,5
4,2,50,118,5
5,2,51,120,11
6,1,56,28,5
7,1,60,78,5
8,2,47,120,9~11
9,2,50,120,11


In [63]:
%timeit
df2 = pd.DataFrame()
for idx, row in df.iterrows():
    
    # Copy row of 1st attempt
    append_row = row.copy(deep=True)
    append_row["solved"] = row["solved_in_1"]
    append_row["skill"] = row["skills"][0]
    df2 = df2.append(append_row)
    
    
    # If 1st attempt was not successful, create row for 2nd attempt
    if row["solved_in_1"] == 2 and len(row["outcomes"]) >= 2:
        append_row = row.copy(deep=True)
        append_row["solved"] = row["solved_in_2"]
        append_row["skill"] = row["skills"][1]
        df2 = df2.append(append_row)
    
    

In [76]:
allowed_skills = [5, 11, 9, 7]
for s in allowed_skills:
    filename = "data_2nd_attempt_skill_%d.csv" % s
    data_2nd_attempt_skill = df2[df2["skill"] == s][["solved", "student_id", "problem_id", "skill"]]
    data_2nd_attempt_skill.to_csv(filename, sep='\t', header=False, index=False)
    print("Data for skill %d" % s)
    print(data_2nd_attempt_skill.groupby("solved").count()["student_id"]/data_2nd_attempt_skill.groupby("solved").count()["student_id"].sum())

Data for skill 5
solved
1.0    0.385042
2.0    0.614958
Name: student_id, dtype: float64
Data for skill 11
solved
1.0    0.280612
2.0    0.719388
Name: student_id, dtype: float64
Data for skill 9
solved
1.0    0.401274
2.0    0.598726
Name: student_id, dtype: float64
Data for skill 7
solved
1.0    0.358885
2.0    0.641115
Name: student_id, dtype: float64


In [131]:
from collections import defaultdict

In [132]:
%timeit
N = 3
SKILL = 5

df_train = pd.DataFrame()
df_test = pd.DataFrame()
outcome_states = {"P": 1, "F": 2}
count_student = defaultdict(int)

for idx, row in df.iterrows():
    # Only get problems with current skill
    if SKILL not in row["skills"]:
        continue
        
    # Count how many problems that student has done
    nP = count_student[row["student_id"]]
    
    # Get the first 3 problems that a student tries to solve
    if nP < 3:
        count_student[row["student_id"]] += 1
        
        # Copy row of 1st attempt
        append_row = row.copy(deep=True)
        append_row["solved"] = row["solved_in_1"]
#         append_row["skill"] = row["skills"][0]
        append_row["skill"] = SKILL
        df_train = df_train.append(append_row)

        # If 1st attempt was not successful, create row for 2nd attempt
        if row["solved_in_1"] == 2 and len(row["outcomes"]) >= 2:
            append_row = row.copy(deep=True)
            append_row["solved"] = row["solved_in_2"]
#             append_row["skill"] = row["skills"][1]
            append_row["skill"] = SKILL
            df_train = df_train.append(append_row)

        # If 2nd attempt was not successful, create row for 3rd attempt
        if row["solved_in_2"] == 2 and len(row["outcomes"]) >= 3:
            append_row = row.copy(deep=True)
            append_row["solved"] = row["solved_in_3"]
#             append_row["skill"] = row["skills"][2]
            append_row["skill"] = SKILL
            df_train = df_train.append(append_row)

        # If 3rd attempt was not successful, create row for last attempt
        if row["solved_in_3"] == 2 and len(row["outcomes"]) >= 4:
            append_row = row.copy(deep=True)
            append_row["solved"] = outcome_states[row["outcomes"][-1]]
#             append_row["skill"] = row["skills"][-1]
            append_row["skill"] = SKILL
            df_train = df_train.append(append_row)
            
    # Get student's next problem
    elif nP == 3:
        count_student[row["student_id"]] += 1
        df_test = df_test.append(row)

In [136]:
df_test

,outcomes,problem_id,skill_in_1,skill_in_2,skill_in_3,skills,solutions,solved_in_1,solved_in_2,solved_in_3,student_id
38,"[F, P]",41.0,5,5,5,"[5, 5]",[# Complete your function using this header\nd...,2.0,1.0,1.0,56.0
44,[F],7.0,5,5,5,[5],[# Complete your function using this header\nd...,2.0,2.0,2.0,50.0
61,"[F, F, F, F, F, F, F, F, F, P]",49.0,5,5,5,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]",[# Complete your function using this header\nd...,2.0,2.0,2.0,60.0
123,"[F, F, F]",76.0,5,5~7,5~7,"[5, 7, 7]",[#Start your python function here\ndef dec2bin...,2.0,2.0,2.0,92.0
192,"[F, F]",49.0,5,5,5,"[5, 5]",[# Complete your function using this header\nd...,2.0,2.0,2.0,102.0
216,"[F, P]",131.0,5,5,5,"[5, 5]",[# Complete your function using this header\nd...,2.0,1.0,1.0,99.0
219,[P],131.0,5,5,5,[5],[# Complete your function using this header\nd...,1.0,1.0,1.0,112.0
226,[P],130.0,5,5,5,[5],[# Complete your function using this header\nd...,1.0,1.0,1.0,121.0
235,[P],41.0,5,5,5,[5],[# Complete your function using this header\nd...,1.0,1.0,1.0,55.0
247,"[F, F, F, F, F, F, F, F, F, F, F, F, F, F, F, P]",117.0,5,5,5,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 9]",[# Complete your function using this header\nd...,2.0,2.0,2.0,64.0


In [137]:
df_test.shape

(17, 11)

In [113]:
df_train[df_train["student_id"] == 56]

,outcomes,problem_id,skill,skill_in_1,skill_in_2,skill_in_3,skills,solutions,solved,solved_in_1,solved_in_2,solved_in_3,student_id
6,[P],28.0,5.0,5,5,5,[5],[#Start your python function here\ndef max_of_...,1.0,1.0,1.0,1.0,56.0
12,"[F, F, F, F, P]",115.0,5.0,5,11~5,11~5,"[5, 11, 11, 11, 11]",[# Complete your function using this header\nd...,2.0,2.0,2.0,2.0,56.0
12,"[F, F, F, F, P]",115.0,11.0,5,11~5,11~5,"[5, 11, 11, 11, 11]",[# Complete your function using this header\nd...,2.0,2.0,2.0,2.0,56.0
12,"[F, F, F, F, P]",115.0,11.0,5,11~5,11~5,"[5, 11, 11, 11, 11]",[# Complete your function using this header\nd...,2.0,2.0,2.0,2.0,56.0
12,"[F, F, F, F, P]",115.0,11.0,5,11~5,11~5,"[5, 11, 11, 11, 11]",[# Complete your function using this header\nd...,1.0,2.0,2.0,2.0,56.0
34,[P],126.0,5.0,5,5,5,[5],[# Complete your function using this header\n#...,1.0,1.0,1.0,1.0,56.0


In [117]:
df_test

,outcomes,problem_id,skill_in_1,skill_in_2,skill_in_3,skills,solutions,solved_in_1,solved_in_2,solved_in_3,student_id
42,"[P, F]",41.0,5,5,5,"[5, 5]",[# Complete your function using this header\nd...,1.0,1.0,1.0,56.0
48,[F],7.0,5,5,5,[5],[# Complete your function using this header\nd...,2.0,2.0,2.0,50.0
61,"[F, F, P]",76.0,5,5,11~5,"[5, 5, 11]",[# Complete your function using this header\nd...,2.0,2.0,1.0,60.0
123,"[F, F, F, F, F]",117.0,5,5,5,"[5, 5, 5, 5, 5]",[# Complete your function using this header\nd...,2.0,2.0,2.0,92.0
192,"[F, F]",49.0,5,5,5,"[5, 5]",[# Complete your function using this header\nd...,2.0,2.0,2.0,102.0
216,"[F, P]",131.0,5,5,5,"[5, 5]",[# Complete your function using this header\nd...,2.0,1.0,1.0,99.0
219,[P],131.0,5,5,5,[5],[# Complete your function using this header\nd...,1.0,1.0,1.0,112.0
226,[P],130.0,5,5,5,[5],[# Complete your function using this header\nd...,1.0,1.0,1.0,121.0
235,[P],41.0,5,5,5,[5],[# Complete your function using this header\nd...,1.0,1.0,1.0,55.0
248,"[F, F, F, F, F, F, F, F, F, F, F, F, F, F, F, P]",117.0,5,5,5,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 9]",[# Complete your function using this header\nd...,2.0,2.0,2.0,64.0


In [138]:
filename = "train.csv"
df_train[["solved", "student_id", "problem_id", "skill"]].to_csv(filename, sep='\t', header=False, index=False)

filename = "test.csv"
df_test[["solved_in_1", "student_id", "problem_id", "skill_in_1"]].to_csv(filename, sep='\t', header=False, index=False)

In [91]:
allowed_skills = [5, 11, 9, 7]
for s in allowed_skills:
    filename = "data_last_attempt_skill_%d.csv" % s
    data_last_attempt_skill = df_train[df_train["skill"] == s][["solved", "student_id", "problem_id", "skill"]]
    data_last_attempt_skill.to_csv(filename, sep='\t', header=False, index=False)
    print("Data for skill %d" % s)
    print(data_last_attempt_skill.groupby("solved").count()["student_id"]/data_last_attempt_skill.groupby("solved").count()["student_id"].sum())

Data for skill 5
solved
1.0    0.375
2.0    0.625
Name: student_id, dtype: float64
Data for skill 11
solved
1.0    0.324324
2.0    0.675676
Name: student_id, dtype: float64
Data for skill 9
solved
1.0    0.39823
2.0    0.60177
Name: student_id, dtype: float64
Data for skill 7
solved
1.0    0.379475
2.0    0.620525
Name: student_id, dtype: float64
